1) define list of wav files, load model

2) for each wav file

    load first segment,
    
        get spectrogram
    
        preprocess, normalize scale...

        ([option] save data! or jump previous step because it has already been saved?)
        
        aply model
        
        get predictions
        
        get first segment annotations
    
    load 2nd segment
    
        get spectrogram
    
        preprocess, normalize scale...

        ([option] save data! or jump previous step because it has already been saved?)
    
        apply model
        
        get predictions
        
        concatenate new annotation to previous anotation file
     
     etc.. till end of wav file
         
     save single anotation file for whole wav.
    
    
3) for each prediction annotation data that becomes available, 

    run code to compute regions files for sonic visualizer!
        
        transform list of prediction matrix per segment [audiox_startSec_to_finishSec, pred_matrix]
        
    this can be done in separate/ parallel to the first step...
    

In [3]:
import numpy as np
import os
import librosa
from tqdm.notebook import tqdm
import csv
import tensorflow as tf
import sys
import pickle
from keras.layers import Input
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers import Reshape, Permute, multiply
from keras.layers import TimeDistributed, Dense, Dropout
from keras.layers import GRU, Bidirectional
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from keras import optimizers
from tensorflow.python.client import device_lib
from sklearn.metrics import roc_curve, auc
from keras import backend as K
import datetime

from keras.models import load_model

In [ ]:
def reformat(x): # modified for single spectrogram. n =1, no labels!
    """Reformat data into a suitable format.
    
    # Arguments
        dataset: dataset in format (id, spectro, label)
        
    # Returns
        x: spectros normalised across each mel band in format (n, timesteps, mel bands, 1)
        y: labels in format (n, timesteps, 8)
    """
#     x = dataset[:, 1] 
#     x = np.stack(x) # reshape to (n, mel bands, timesteps)
    x = np.expand_dims(np.moveaxis(x, 1, -1), axis=3) # reformat x to (n, timesteps, mel bands, 1)  
#     y = dataset[:, 2] 
#     y = np.moveaxis(np.stack(y), 1, -1) # reformat y to (n, timesteps, 8)
    return x


In [ ]:
segment_size = 6
slide = 6
mels = 64
sample_rate = 22_050
timesteps = 259 # number of frames in a segment! =((SR* segment_size)/ hop_length)
timesteps_per_second = timesteps / segment_size

############# settings: ##################################################################
save_path =  #where to save the prediction matrix to
wavs_list = [] # list of wav files to make annotations
model_path =   # which model to use (path)

prediction_theshold = 0.5

In [ ]:


# load model:

model = load_model(model_path)
#


for wfile in wavs_list:
    prediction_annotations = []
    
    print('load audio')
    #Load audio,
    y, sr = librosa.load(wfile, sr=sample_rate)
    length = int(len(y) / sr) #seconds
    remainder = length % segment_size
    #     import pdb; pdb.set_trace()

    audio_id = wfile[:-4]
    
#   segment, cycle for each segment:
    for t in tqdm(range(0, length - remainder - segment_size, slide),
                  desc='create_spectros'):
        start = t
        stop = t + segment_size
        print('start in frames', str(sr*start))
        print('stop in frames: ', str((sr*stop)))
        current_y = y[sr*start:(sr*stop)]
        
        # create spectrogram
        spectro = librosa.feature.melspectrogram(y=current_y, sr=sr, n_mels=mels,
                                                 fmax=sr/2)
        dt_id = audio_id + '_' + str(start) + 's_to' + str(stop) + 's'
        print(dt_id)
        
#       treat segment spectrogram for network
#          PCEN:
        spectro_PCEN = librosa.pcen(spectro * (2**31))
#         further normalize, scale?
        x = reformat(spectro_PCEN)
#          downsample mels???? uhh, didnt realize we were doing this... mels = 45
        x = x[:mels]
        
        
        
#          predict with loaded model
        predicted = model.predict(x) 
#          get binary predictions accordingly to threshold
        predicted[predicted > prediction_theshold] = 1
        predicted[predicted <= prediction_theshold] = 0
        
        prediction_annotations.append([dt_id, predicted])
        
    #save data processed for this file
    d_array = np.asarray(prediction_annotations)
    np.save(save_path+audio_id, d_array)
  